In [1]:
import os
import matplotlib.pyplot as pl
import numpy as np
%matplotlib inline
import pandas as pd

from __future__ import division, print_function
import os
import requests
from io import StringIO, BytesIO

from scipy.stats import gamma

from detection_efficiency_functions import *
from data_wrangling import *
from plotting import *

The completeness functions that are specific to 2 dimensions.

In [2]:
def get_completeness(star, period, rp, e, cdpp_cols, cdpp_vals, pgam, mesthres_cols, 
                     mesthres_vals, with_geom=True):
    """
    A helper function to combine all the completeness effects.
    
    :param star:      a pandas row giving the stellar properties
    :param period:    the period in days
    :param c_grid:    the stellar property grid
    :param c:         the stellar property of this star.
    :param rp:        the planet radius in Earth radii
    :param e:         the orbital eccentricity
    :param with_geom: include the geometric transit probability?
    
    """
    aor = get_a(period, star.mass) / star.radius
    pdet = get_pdet(star, aor, period, rp, e, cdpp_cols, cdpp_vals, pgam, mesthres_cols, 
                    mesthres_vals)
    pwin = get_pwin(star, period)
    if not with_geom:
        return pdet * pwin
    pgeom = get_pgeom(aor, e)
    return pdet * pwin * pgeom


def calc_completeness_for_each_star(ranges, grid_sizes, cdpp_cols, cdpp_vals, pgam, mesthres_cols, 
                                    mesthres_vals):    
    x = np.linspace(ranges[0][0], ranges[0][1], grid_sizes[0])
    y = np.linspace(ranges[1][0], ranges[1][1], grid_sizes[1])
    xgrid, ygrid = np.meshgrid(x, y, indexing="ij")
    comp = np.zeros_like(xgrid)
    for _, star in stlr.iterrows():
        comp += get_completeness(star, xgrid, ygrid, 0.0, cdpp_cols, cdpp_vals, pgam, mesthres_cols, 
                                 mesthres_vals with_geom=True)
    return comp/len(stlr), xgrid, ygrid

SyntaxError: invalid syntax (<ipython-input-2-a67e8615365d>, line 33)

In [ ]:
stlr = get_catalog("q1_q16_stellar")

cdpp_cols = [k for k in stlr.keys() if k.startswith("rrmscdpp")]
cdpp_vals = np.array([k[-4:].replace("p", ".") for k in cdpp_cols], dtype=float)

# Pre-compute and freeze the gamma function from Equation (5) in
# Burke et al.
pgam = gamma(4.65, loc=0., scale=0.98)
mesthres_cols = [k for k in stlr.keys() if k.startswith("mesthres")]
mesthres_vals = np.array([k[-4:].replace("p", ".") for k in mesthres_cols],
                         dtype=float)


var_names = ["koi_period", "koi_prad"]
period_rng = (50, 300)
rp_rng = (0.75, 2.5)

ranges = [period_rng, rp_rng]
stlr, kois = selection(stlr, ranges)

nbins = [9, 11]
bins = plot_obs_and_get_bin_edges(kois, var_names, nbins, ranges)

Calculate the completeness over period and radius.

In [ ]:
grid_points_per_bin = 2
comp, period_grid, rp_grid = calc_completeness_for_each_star(ranges, nbins*grid_points_per_bin, 
                                                             cdpp_cols, cdpp_vals, pgam, mesthres_cols, 
                                                             mesthres_vals)